In [123]:
import numpy as np
from tensorflow.keras.datasets import mnist
from cvxopt import matrix, solvers
import time

In [124]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [125]:
# Print the shapes of the loaded data (there is a lot of checking in this part)
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print("x_test shape:", x_test.shape)
print("y_test shape:", y_test.shape)

x_train shape: (60000, 28, 28)
y_train shape: (60000,)
x_test shape: (10000, 28, 28)
y_test shape: (10000,)


In [126]:

def filter_digits(images, labels, digit_list):
    mask = np.isin(labels, digit_list)
    return images[mask], labels[mask]

digits = [2, 3, 8, 9]
x_train, y_train = filter_digits(x_train, y_train, digits)
x_test, y_test = filter_digits(x_test, y_test, digits)


print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print("x_test shape:", x_test.shape)
print("y_test shape:", y_test.shape)

x_train shape: (23889, 28, 28)
y_train shape: (23889,)
x_test shape: (4025, 28, 28)
y_test shape: (4025,)


In [127]:
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784) 

In [128]:

def create_subset(x_data, y_data, subset_size=4000):
    indices = np.random.choice(np.arange(len(x_data)), size=subset_size, replace=False)
    x_subset = x_data[indices]
    y_subset = y_data[indices]
    return x_subset, y_subset


x_train_subset, y_train_subset = create_subset(x_train, y_train, 4000)
x_test_subset, y_test_subset = create_subset(x_test, y_test, 2000)

# Check shapes before processing
print("x_train_subset shape:", x_train_subset.shape)
print("y_train_subset shape:", y_train_subset.shape)
print("x_test_subset shape:", x_test_subset.shape)
print("y_test_subset shape:", y_test_subset.shape)


x_train_subset shape: (4000, 784)
y_train_subset shape: (4000,)
x_test_subset shape: (2000, 784)
y_test_subset shape: (2000,)


In [129]:
# Convert labels for SVM format (+1, -1 for binary classification)
y_train_subset = np.where(np.isin(y_train_subset, [2, 3]), -1, 1)
y_test_subset = np.where(np.isin(y_test_subset, [2, 3]), -1, 1)
print("y_test_subset shape:", y_test_subset.shape)

y_test_subset shape: (2000,)


In [130]:
def svm_primal_qp(X, y, C=1.0):
    m, n = X.shape 
    P = matrix(np.block([
        [np.eye(n), np.zeros((n, m))],
        [np.zeros((m, n + m))]
    ]))
    q = matrix(np.concatenate([
        np.zeros(n),
        C * np.ones(m)
    ]))
    G = matrix(np.vstack([
        np.hstack([-np.diag(y) @ X, -np.eye(m)]),
        np.hstack([np.zeros((m, n)), -np.eye(m)])
    ]))
    h = matrix(np.concatenate([-np.ones(m), np.zeros(m)]))
    solution = solvers.qp(P, q, G, h)
    return solution, n

def predict(X, w, b):
    return np.sign(np.dot(X, w) + b)

C_values = [0.01, 0.1, 1, 10, 100]
train_accuracies = []
test_accuracies = []

start_time = time.time()

for C in C_values:
    solution, n = svm_primal_qp(x_train_subset, y_train_subset, C=C)
    w = np.array(solution['x'][:n]).reshape(-1)
    b = 0  

    # Calculate training accuracy
    train_predictions = predict(x_train_subset, w, b)
    train_accuracy = np.mean(train_predictions == y_train_subset)
    train_accuracies.append(train_accuracy)

    # Calculate test accuracy
    test_predictions = predict(x_test_subset, w, b)
    test_accuracy = np.mean(test_predictions == y_test_subset)
    test_accuracies.append(test_accuracy)
    
end_time = time.time()
training_time = end_time - start_time

# Print the results to observe the best C
for C, train_acc, test_acc in zip(C_values, train_accuracies, test_accuracies):
    print(f"C: {C}, Training Accuracy: {train_acc:.3f}, Test Accuracy: {test_acc:.3f}")
print(f"Training time: {training_time:.2f} seconds")

     pcost       dcost       gap    pres   dres
 0:  4.0410e+00  5.2329e+02  3e+04  3e+00  3e+06
 1:  8.2302e+01 -5.2030e+02  6e+02  5e-02  6e+04
 2:  6.1103e+01 -6.9569e+01  1e+02  8e-03  9e+03
 3:  3.6579e+01 -2.6383e+01  6e+01  3e-03  4e+03
 4:  1.9632e+01 -1.0617e+01  3e+01  2e-03  2e+03
 5:  1.2809e+01 -6.3998e+00  2e+01  9e-04  1e+03
 6:  7.7690e+00 -3.5545e+00  1e+01  5e-04  5e+02
 7:  5.9430e+00 -2.9143e+00  9e+00  3e-04  4e+02
 8:  4.7012e+00 -2.5670e+00  7e+00  2e-04  3e+02
 9:  3.7702e+00 -2.3450e+00  6e+00  2e-04  2e+02
10:  3.2873e+00 -2.2996e+00  6e+00  1e-04  2e+02
11:  3.0317e+00 -2.2969e+00  5e+00  1e-04  1e+02
12:  2.9034e+00 -2.2631e+00  5e+00  1e-04  1e+02
13:  2.4242e+00 -1.8408e+00  4e+00  6e-05  7e+01
14:  1.0628e+00 -6.8036e-01  2e+00  2e-05  3e+01
15:  6.8829e-01 -3.7373e-01  1e+00  1e-05  1e+01
16:  2.3635e-01 -7.2400e-02  3e-01  2e-06  3e+00
17:  9.7660e-02  4.5709e-03  9e-02  6e-07  8e-01
18:  6.5038e-02  2.4195e-02  4e-02  1e-07  2e-01
19:  4.6131e-02  3.43